# Load DNS and mDNS datasets

In [1]:
import os
import sys

import numpy as np
import pandas as pd

module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch_geometric.transforms as T

from libs.loader import DNS
from libs.utils import score

## Load Graphs

In [2]:
kg_path = lambda graph_name: f'./data/{graph_name}'

#### mDNS

In [3]:
# dataset = DNS(root=kg_path('mDNS'), transform=T.Compose([T.NormalizeFeatures(), T.ToUndirected()]), balance_gt=True)
# data = dataset[0]
# data['domain_node']['test_mask'].unique(return_counts=True)

#### DNS

In [4]:
dataset = DNS(root=kg_path('DNS'), transform=T.Compose([T.NormalizeFeatures(), T.ToUndirected()]))
data = dataset[0]
data['domain_node']['test_mask'].unique(return_counts=True)

Remove parallel edges: type
similar    50910
dtype: int64


(tensor([False,  True]), tensor([366174,   7301]))

In [5]:
homo_data = dataset.to_homogeneous()

homo_data

Data(
  node_type=[447068],
  x=[447068, 12],
  edge_index=[2, 897588],
  edge_type=[897588],
  train_mask=[447068],
  val_mask=[447068],
  test_mask=[447068],
  y=[447068],
  edge_map={
    0=[2],
    1=[2],
    2=[2]
  },
  num_nodes=447068
)

In [6]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GATConv, HeteroConv, Linear


class HeteroGNN(torch.nn.Module):
    def __init__(self, metadata, hidden_channels, out_channels, num_layers):
        super().__init__()

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HeteroConv({
                edge_type: GATConv((-1,-1), hidden_channels)
                for edge_type in metadata[1]
            })
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
            x_dict = {key: F.leaky_relu(x) for key, x in x_dict.items()}
        return self.lin(x_dict['domain_node'])
    

model = HeteroGNN(data.metadata(), hidden_channels=64, out_channels=2,
                  num_layers=2)

In [7]:
cuda_device = 3
torch.manual_seed(42)

if torch.cuda.is_available():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(cuda_device)

    data, model = data.to(device), model.to(device)

with torch.no_grad():  # Initialize lazy modules.
    out = model(data.x_dict, data.edge_index_dict)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)


def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)
    mask = data['domain_node'].train_mask
    loss = F.cross_entropy(out[mask], data['domain_node'].y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict).argmax(dim=-1)

    accs = []
    for split in ['train_mask', 'val_mask', 'test_mask']:
        mask = data['domain_node'][split]
        acc = (pred[mask] == data['domain_node'].y[mask]).sum() / mask.sum()
        accs.append(float(acc))
    return accs


for epoch in range(1, 201):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 20 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, '
              f'Val: {val_acc:.4f}, Test: {test_acc:.4f}')
    
model.eval()
with torch.no_grad():
    pred = model(data.x_dict, data.edge_index_dict).argmax(dim=-1)
mask = data['domain_node']['test_mask']
scores = score(pred[mask],data['domain_node'].y[mask])
for metric, score in scores.items():
    print(metric, ':{:.2f}'.format(score))

Epoch: 020, Loss: 0.3181, Train: 0.8657, Val: 0.8704, Test: 0.8651
Epoch: 040, Loss: 0.2433, Train: 0.8923, Val: 0.8835, Test: 0.8888
Epoch: 060, Loss: 0.2262, Train: 0.8961, Val: 0.8876, Test: 0.8925
Epoch: 080, Loss: 0.2077, Train: 0.9109, Val: 0.9053, Test: 0.9080
Epoch: 100, Loss: 0.1819, Train: 0.9269, Val: 0.9248, Test: 0.9234
Epoch: 120, Loss: 0.1668, Train: 0.9373, Val: 0.9336, Test: 0.9299
Epoch: 140, Loss: 0.1501, Train: 0.9436, Val: 0.9408, Test: 0.9381
Epoch: 160, Loss: 0.1431, Train: 0.9487, Val: 0.9491, Test: 0.9397
Epoch: 180, Loss: 0.1405, Train: 0.9496, Val: 0.9491, Test: 0.9403
Epoch: 200, Loss: 0.1391, Train: 0.9503, Val: 0.9495, Test: 0.9410
tn, fp, fn, tp 1068 264 167 5802
tn, fp, fn, tp 1068 264 167 5802
acc :0.94
f1 :0.94
auc :0.89
prec :0.96
recall :0.97
fpr :0.20
